In [1]:
# !pip install tensorflow==1.15.5

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import tensorflow as tf
import time

In [3]:
def load(input_file):
    data = open(input_file, 'r').read()   
    chars = sorted(set(data))
    vocab_size = len(chars)
    ch2ix = {ch: i for i, ch in enumerate(chars)}
    ix2ch = {i: ch for i, ch in enumerate(chars)}
    return data, ch2ix, ix2ch, vocab_size

In [4]:
def encode(seq, vocab_size):                            #return 1 hot encoding of sequences
    enc = np.zeros((1, vocab_size), dtype=int)
    enc[0][seq[0]] = 1
    for i in range(1, len(seq)):
        row = np.zeros((1, vocab_size), dtype=int)
        row[0][seq[i]] = 1
        enc = np.append(enc, row, axis=0)
    return enc
"""
seq=[1,2,3]
vocab_size=5
encode(seq,vocab_size)
Output:
array([[0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0]])
"""

'\nseq=[1,2,3]\nvocab_size=5\nencode(seq,vocab_size)\nOutput:\narray([[0, 1, 0, 0, 0],\n       [0, 0, 1, 0, 0],\n       [0, 0, 0, 1, 0]])\n'

In [5]:
def gen(data, seq_length, char_to_idx, vocab_size, p=0):
  """
  Yield Generated inputs and targets from data of size seq_length
  """
  p = int(p)   #starting pointer
  while True:
      if p + seq_length + 1 >= len(data):
          print("Till: ", p, len(data))
          p = 0  # go to start of data
      inputs = [char_to_idx[char] for char in data[p: p + seq_length]]  # Sequence of inputs (numbers)
      target = [char_to_idx[char] for char in data[1 + p: 1 + p + seq_length]]
      inputs = np.expand_dims(encode(inputs, vocab_size), axis=1)  # shape: (seq_length, input_dim)
      targets = np.expand_dims(encode(target, vocab_size), axis=1)
      p = p + seq_length
      yield inputs, targets

In [6]:
def plot(loss, smooth_loss, it, it_per_epoch, base_name=''):
    fig = plt.figure(figsize=(10, 5), dpi=100)
    plt.plot(loss)
    plt.plot(smooth_loss)
    epochs = [i * int(it_per_epoch) for i in range(int(it / it_per_epoch) + 1)]
    plt.plot(epochs, [smooth_loss[i] for i in epochs], linestyle='', marker='o')
    print([smooth_loss[i] for i in epochs])
    plt.title('Loss')
    plt.xlabel('Iteration')
    plt.ylim([0, 5])
    if base_name != '':
        fig.savefig(base_name + '_plot.png')
    else:
        plt.show()
    plt.close("all")

In [7]:
def plot_vars(var1, var2, var3, var4, it, it_per_epoch, base_name=''):
    fig = plt.figure(figsize=(12, 5), dpi=100)
    plt.plot(var1, label = "rnn/lstm_cell/kernel:0")
    plt.plot(var2, label = "Variable:0")
    plt.plot(var3, label = "rnn/lstm_cell/bias:0")
    plt.plot(var4, label = "Variable_1:0")
    epochs = [i * int(it_per_epoch) for i in range(int(it / it_per_epoch) + 1)]
    plt.plot(epochs, [var2[i] for i in epochs], linestyle='', marker='o')
    plt.title('Loss')
    plt.xlabel('Iteration')
    plt.legend(loc='upper right', shadow=True, fontsize='x-small', bbox_to_anchor=(1.1, 1))
    plt.ylim([-10, 10])
    if base_name != '':
        fig.savefig(base_name + '_plot.png')
    else:
        plt.show()
    plt.close("all")

In [8]:
# Function to sample text using a model trained on a certain text corpus
def sample(sample_length, session):
    seed = encode([int(np.random.uniform(0, vocab_size))], vocab_size)
    seed = np.array([seed])
    _char_state = np.zeros((2, batch_size, hidden_dim))
    txt = ''
    for i in range(sample_length):
        char_probs, _char_state = session.run([probabilities, current_state],
                                           feed_dict={x: seed, init_state: _char_state})
        pred = np.random.choice(range(vocab_size), p=char_probs[0])
        seed = np.expand_dims(encode([pred], vocab_size), axis=0)
        character = idx_to_char[pred]
        txt += character
    return txt

In [9]:
# hyperparameters
learning_rate = 1e-2
seq_length = 50
hidden_dim = 500
batch_size = 1

In [10]:
# load data
data_name = 'hello'
input_file = data_name +'.txt'
data, char_to_idx, idx_to_char, vocab_size = load(input_file)
print('data has %d characters, %d unique.' % (len(data), vocab_size))
print("First 10 characters are: ",[idx_to_char[i] for i in range(10)])

data has 1061 characters, 37 unique.
First 10 characters are:  ['\n', ' ', '"', ',', '-', '.', 'A', 'D', 'L', 'M']


In [11]:
# Create data generator
data_feed = gen(data, seq_length, char_to_idx, vocab_size)

In [12]:
# TensorFlow input variables
x = tf.placeholder("float", [None, batch_size, vocab_size], name="x")
y = tf.placeholder("float", [None, batch_size, vocab_size], name="y")
init_state = tf.placeholder(tf.float32, [2, batch_size, hidden_dim], name="init_state")
# model architecture
# lstm layer
lstm_cell = tf.nn.rnn_cell.LSTMCell(hidden_dim, state_is_tuple=True, name="Cell")
rnn_tuple_state = tf.nn.rnn_cell.LSTMStateTuple(init_state[0], init_state[1])
# dense layer parameters
dense_weights = tf.get_variable("out_w", shape=[hidden_dim, vocab_size])
dense_bias = tf.get_variable("out_b", shape=[vocab_size])

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [13]:
# model
h_states, current_state = tf.nn.dynamic_rnn(lstm_cell, x, initial_state=rnn_tuple_state,
                                            time_major=True, dtype=tf.float32)
logits = tf.matmul(h_states[:, 0, :], dense_weights) + dense_bias
probabilities = tf.nn.softmax(logits, name="probabilities")

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
# model evaluation
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits)
loss = tf.reduce_mean(cross_entropy, name="loss")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training = optimizer.minimize(loss, name="training")

In [15]:
# Defining variables initializer
init_op = tf.global_variables_initializer()

In [16]:
# Bookkeeping variables
save_path = 'tensor_model/'
loss_hist = [-np.log(1.0 / vocab_size)]  # loss at iteration 0
smooth_loss = loss_hist.copy()
it = 0
it_per_epoch = len(data) / seq_length
p = (it % it_per_epoch) * seq_length
elapsed_time = 0
start = time.time()

# Saver
saver = tf.train.Saver()
restore = False

# Training
with tf.Session() as sess:
    if restore:
        saver.restore(sess, save_path + "model")
    else:
        sess.run(init_op)
    _current_state = np.zeros((2, batch_size, hidden_dim))
    for p in range(501):
        # show progress and bookkeeping
        if p % 100 == 0:
            print('\niter %d, loss: %f' % (p, smooth_loss[-1]))  # print progress
            print(sample(50, sess))
            saver.save(sess, save_path + 'model')
            plot(loss_hist, smooth_loss, it, it_per_epoch, base_name=save_path + "tensor")

        # collect data for next step
        inputs, targets = (next(data_feed))
        _loss, _, _current_state = sess.run([loss, training, current_state],
                                        feed_dict={x: inputs,
                                                   y: targets,
                                                   init_state: _current_state})
        loss_hist.append(_loss)
        smooth_loss.append(smooth_loss[-1] * 0.999 + loss_hist[-1] * 0.001)

end = time.time()
print("      Training time: ", end - start, "\n")


iter 0, loss: 3.610918
jii.eunaafto",ldDagi"znDituit
.mMhLtNkThlvuwMlxp"j
[3.6109179126442243]
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061

iter 100, loss: 3.531884
cv"ry an huatonalhe vam mapy siinisg mansworarbnal
[3.6109179126442243]
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061

iter 200, loss: 3.369614
t or thegienal, theoheof Mapps hay mex dimevef may
[3.6109179126442243]
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061

iter 300, loss: 3.145267
he world.tgeographic relistolyabitwo dimensional, 
[3.6109179126442243]
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061

iter 400, loss: 2.888933
n mapping, or communter ne worlitony long troditio
[3.6109179126442243]
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061
Till:  1050 1061

iter 500, loss: 2.627409
gy mapping. The space being mapping, DNA mapping, 
[3.6109179126442243]
      Training time:  8

In [17]:
# Restore
print("Restoring the trained variables\n")
saver = tf.train.Saver()
with tf.Session() as sess:
    # If saver.restore(...) commented -> 'Error: Attempting to use uninitialized value out_w'
    # can be corrected with sess.run(init_op), but training is forgotten
    saver.restore(sess, save_path + "model")
    print(sample(600, sess))

Restoring the trained variables

INFO:tensorflow:Restoring parameters from tensor_model/model

Although the earliest maps yneys abe static, fixed to paper or some other durable medium, while others are dynamic or inoeractive. Although most commonly used to depict geog aphy, maps may represent tno othr or some space, such as in brain mapping, DNA mapping. The space being mapped may be two nimens are abstract spac s of the intirior of the surface of the earth, or even more abstract spaces of any dimension, such as arise in modeling phenomena having many independent variables.
Although the earliest maps known are of the heavens, geographic maps or territory have a very long tradition and 
